# 3. Deep Hedging Model

In this notebook, we implement and train a neural network that learns to hedge a European call option dynamically using simulated market data.

> **Goal.** Train a trading policy $(a_t)$ that minimizes **tail risk** (CVaR) of the terminal hedging error
> $$
X \;=\; -Z_T \;+\; \sum_{t=0}^{T-1} a_t\,\Delta S_t \;-\; \underbrace{\gamma\sum_{t=0}^{T-1} S_t\,|a_t - a_{t-1}|}_{\text{transaction cost}}
 $$
> on simulated paths from Notebook 2. We do this end-to-end with a recurrent policy network, a symbolic rollout, and a Rockafellar–Uryasev (RU) CVaR head.

---

## 3.1) What this notebook consumes / produces

**Inputs (from Notebook 2):**
- `data/S_train.npy`, `S_val.npy`, `S_test.npy`  — each shape `(N, n_steps+1)`
- `data/Z_T_train.npy`, `Z_T_val.npy`, `Z_T_test.npy`  — each shape `(N,)`
- `data/meta.json` — `S0`, `K`, `T`, `n_steps`, `dt`, plus knobs like `alpha`, `gamma`, `h_max` (optional)

**Outputs (used by Notebook 4):**
- Trained weights (best by tail metrics)
- Arrays for backtesting: $(X)$, $(\text{PnL})$, $(\text{cost})$, $(\text{turnover})$, $(Z_T)$, and optionally positions $(a_t)$.

---

## 3.2) Setup & reproducibility

**What to do here:**
- Import `numpy`, `tensorflow` (Keras), `json`, `pathlib`.
- Fix seeds (`tf.keras.utils.set_random_seed(SEED)`).
- Read `meta.json`, unpack: `S0, K, T_years, n_steps, dt`.  
- Choose training knobs (typical defaults shown; adjust later in §12):
  - `ALPHA = 0.90` (CVaR level)
  - `GAMMA = 0.001` (turnover cost scale; ≈10 bps)
  - `H_MAX = 5` (position clamp \(|a_t|\le H_{\max}\))
  - `BATCH`, `EPOCHS`, `LR`
- Load arrays `S_*` and `Z_T_*` and print shapes (sanity).

In [1]:
# ============================================================
# Notebook 3 (Keras) — Deep Hedging with CVaR Utility
# ============================================================
# Requirements: TensorFlow 2.x (tf.keras), numpy, Keras 3
# Inputs:   data/S_train.npy, S_val.npy, S_test.npy
#           data/Z_T_train.npy, Z_T_val.npy, Z_T_test.npy
#           data/meta.json  (S0, K, T, n_steps, dt, alpha, etc.)
# Output:   results/hedging_eval_test_keras.npz  (keys: V_T, Z_T)
# ============================================================

import os, json, math, time
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # quiet TF logs

from pathlib import Path
import numpy as np
import tensorflow as tf

# --------------------------
# 0) Repro + folders
# --------------------------
SEED = 42
tf.keras.utils.set_random_seed(SEED)

DATA_DIR = Path("data")
RESULTS_DIR = Path("results"); RESULTS_DIR.mkdir(exist_ok=True)

# --------------------------
# 1) Load meta + raw arrays
# --------------------------
with open(DATA_DIR/"meta_b.json", "r") as f:
    meta = json.load(f)

S0       = float(meta["S0"])
K        = float(meta["K"])
T_years  = float(meta["T"])
n_steps  = int(meta["n_steps"])
dt       = float(meta["dt"])
n_feat   = 4 

# --------------------------
# 2) Training knobs
# --------------------------
ALPHA  = 0.9 # CVaR tail level
GAMMA  = 0.0005 # Transaction cost scale γ = 0.001 (10 bps per unit turnover)
H_MAX  = 5 # max abs(position)
BATCH  = 400 # for ALPHA=0.95 ~ 400
EPOCHS = 200
LR     = 1e-3

S_train = np.load(DATA_DIR/"S_train_b.npy")   # (N_train, n_steps+1)
S_val   = np.load(DATA_DIR/"S_val_b.npy")
S_test  = np.load(DATA_DIR/"S_test_b.npy")

Z_T_train = np.load(DATA_DIR/"Z_T_train_b.npy").astype(np.float32)
Z_T_val   = np.load(DATA_DIR/"Z_T_val_b.npy").astype(np.float32)
Z_T_test  = np.load(DATA_DIR/"Z_T_test_b.npy").astype(np.float32)

print(f"Loaded: S_train {S_train.shape}, S_val {S_val.shape}, S_test {S_test.shape}")
print(f"Payoffs: Z_T_train {Z_T_train.shape}, Z_T_val {Z_T_val.shape}, Z_T_test {Z_T_test.shape}")
print(f"n_steps = {n_steps}, N_train = {S_train[:,0].shape}, N_val = {S_val[:,0].shape}, N_test = {S_test[:,0].shape}")

Loaded: S_train (8000, 253), S_val (5000, 253), S_test (5000, 253)
Payoffs: Z_T_train (8000,), Z_T_val (5000,), Z_T_test (5000,)
n_steps = 252, N_train = (8000,), N_val = (5000,), N_test = (5000,)


## 3.3) Build *causal* features and $(\Delta S)$

**What to do here:**
- From each `S` (shape `(N, n_steps+1)`) compute:
  - $(S_t := S[:, 0:n\_steps])$
  - $(S_{t+1} := S[:, 1:n\_steps+1])$
  - $(\Delta S_t = S_{t+1} - S_t)$ → shape `(N, n_steps, 1)`
- Causal features per time step $(t)$ (shape `(N, n_steps, 4)`):
  1. `price_norm`: $(S_t / S_0)$
  2. `tau`: linear grid $(\tau_t = (T - t)/T)$
  3. `logR_past`: **lagged** log return; at $(t=0)$ set to `0`, for $(t>0)$ use $(\log S_t - \log S_{t-1})$
  4. `vol_ann`: rolling std of `logR_past` over a window (e.g., 20), annualized by $(\sqrt{1/dt})$

In [2]:
# --------------------------
# 3) Causal features: [price_norm, tau, log_return, realized_vol_annualized]
# --------------------------
def build_features_from_S(S, S0, n_steps, dt, win=20):
    N = S.shape[0]
    St      = S[:,:-1]                # (N, n_steps)
    St_p1 = S[:, 1:]
    
    dSt = (St_p1 - St)[:,:,np.newaxis].astype(np.float32)
    price_norm = St / S0

    r_next = np.diff(np.log(S), axis=1).astype(np.float32)
    logR_past = np.zeros_like(r_next, dtype=np.float32)     # (N, n_steps)
    if n_steps > 1:
        logR_past[:, 1:] = r_next[:, :-1]

    tau_vec = np.linspace(1.0 - 1.0/n_steps, 0.0, n_steps, dtype=np.float32)
    tau     = np.broadcast_to(tau_vec, (N, n_steps))

    vol = np.empty_like(logR_past, dtype=np.float32)
    for t in range(n_steps):
        l = max(0, t - win + 1)
        w = logR_past[:, l:t+1]
        vol[:, t] = w.std(axis=1)
    steps_per_year = int(round(1.0 / dt))
    vol_ann = vol * np.sqrt(steps_per_year)

    X = np.stack([price_norm, tau, logR_past, vol_ann], axis=-1).astype(np.float32)
    return dSt, X, logR_past.astype(np.float32), vol_ann.astype(np.float32)

dSt_train, X_train, logR_train, vol_train = build_features_from_S(S_train, S0, n_steps, dt, win=20)
dSt_val, X_val,   logR_val,   vol_val   = build_features_from_S(S_val,   S0, n_steps, dt, win=20)
dSt_test, X_test,  logR_test,  vol_test  = build_features_from_S(S_test,  S0, n_steps, dt, win=20)

Z_T_train = np.asarray(Z_T_train, dtype=np.float32).reshape(-1)
Z_T_val   = np.asarray(Z_T_val,   dtype=np.float32).reshape(-1)
Z_T_test  = np.asarray(Z_T_test,  dtype=np.float32).reshape(-1)

print("Feature tensor shapes:",
      "\n dSt_train:", dSt_train.shape,
      "\n  X_train:", X_train.shape,
      "\n  X_val:  ", X_val.shape,
      "\n  X_test: ", X_test.shape,
      "\n  Z_train:", Z_T_train.shape,
      "\n  Z_val:  ", Z_T_val.shape,
      "\n  Z_test: ", Z_T_test.shape)

Feature tensor shapes: 
 dSt_train: (8000, 252, 1) 
  X_train: (8000, 252, 4) 
  X_val:   (5000, 252, 4) 
  X_test:  (5000, 252, 4) 
  Z_train: (8000,) 
  Z_val:   (5000,) 
  Z_test:  (5000,)


In [3]:
n_train  = X_train.shape[0]
n_val  = X_val.shape[0]
n_test  = X_test.shape[0]

## 3.4) Build `tf.data` pipelines

**What to do here:**
- Construct dictionaries per split:
  - `inputs = {"features": X, "dS": dS, "Z_T": Z_T}`  
    (keep `Z_T` **1D** `(N,)`
  - `labels = zeros(N)`  (dummy; our loss reads from the graph)
- Create datasets:
  - `train_ds = Dataset.from_tensor_slices((inputs, labels)).cache().shuffle(...).batch(BATCH).prefetch(AUTOTUNE)`
  - Similarly for `val_ds` (no shuffle) and `test_ds`.
- **Peek a batch** and print dtypes/shapes to confirm:
  - `features: (BATCH, n_steps, 4)`
  - `dS:       (BATCH, n_steps, 1)`
  - `Z_T:      (BATCH,)`

In [4]:
# --------------------------
# 4) Datasets
# --------------------------

inputs = {
    "features": X_train[:,:,:],
    "dS": dSt_train[:,:,:],
    "Z_T": Z_T_train[:]
}

val_inputs = {
    "features": X_val[:,:,:],
    "dS": dSt_val[:,:,:],
    "Z_T": Z_T_val[:]
}

test_inputs = {
    "features": X_test[:,:,:],
    "dS": dSt_test[:,:,:],
    "Z_T": Z_T_test[:]
}

input_label = np.zeros((n_train),dtype=np.float32)
val_label = np.zeros((n_val),dtype=np.float32)
test_label = np.zeros((n_test),dtype=np.float32)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = (tf.data.Dataset.from_tensor_slices((inputs, input_label))
            .cache().shuffle(8192, seed=SEED, reshuffle_each_iteration=True)
            .batch(BATCH).prefetch(AUTOTUNE))

val_ds   = (tf.data.Dataset.from_tensor_slices((val_inputs, val_label))
            .cache().batch(BATCH).prefetch(AUTOTUNE))

test_ds  = (tf.data.Dataset.from_tensor_slices((test_inputs, test_label))
            .batch(BATCH).prefetch(AUTOTUNE))

## 3.5) Zero-hedge baseline (validation)

**What to do here:**
- Baseline wealth if you **do not trade**: $(X^{(0)} = -Z_T)$.
- Compute on **all** validation paths:
  - `mean_baseline = mean(X0)`
  - `VaR_α` and `CVaR_α` at `ALPHA` (note: sort ascending, take worst tail)

---

In [5]:
# --------------------------
# 5) Baseline 
# --------------------------
wealth = - Z_T_val

val_mean_baseline  = wealth.mean()
sorted_wealth = np.sort(wealth)

k = int(np.ceil((1 - ALPHA) * sorted_wealth.size))
values = sorted_wealth[:k]

val_cvar_baseline = values.mean()
val_p05_baseline = np.percentile(wealth, 5)

print(f"val_cvar_baseline: {val_cvar_baseline:.6f}")
print(f"val_mean_baseline: {val_mean_baseline:.6f}")
print(f"val_p05_baseline: {val_p05_baseline:.6f}")

val_cvar_baseline: -79.111267
val_mean_baseline: -14.786800
val_p05_baseline: -68.168206


## 3.6) Policy network (backbone)

**What to do here:**
- Define three **Inputs**:
  - `features`: `(n_steps, 4)` — the only tensor used by the policy
  - `dS`: `(n_steps, 1)` — used *later* in rollout, not inside the policy
  - `Z_T`: `()` — scalar terminal payoff, used *later* in rollout, not inside the policy
- Build a small recurrent backbone, e.g. GRU→GRU→Dense:
  - GRU(32, return_sequences=True) → GRU(16, return_sequences=True) → Dense(32, relu)
  - Final Dense to 1 → raw position `a_raw(t)`
  - **Clamp**: `a(t) = H_MAX * tanh(a_raw(t))` to bound $(|a_t|\le H_{\max})$

**Why these choices:**
- GRUs capture temporal context without heavy complexity.
- `tanh` clamp stabilizes training, enforces realistic position bounds.
- `dS` and `Z_T` must *not* leak into the policy — they are used only in P&L computation.

---

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
# --------------------------
# 6) Model
# --------------------------

features_in = keras.Input(shape=(n_steps, n_feat), name="features")
dS_in = keras.Input(shape=(n_steps,1), name="dS")
Z_T_in = keras.Input(shape=(), name="Z_T")

H = layers.GRU(32, return_sequences=True, name="gru_32")(features_in)
H = layers.GRU(16, return_sequences=True, name="gru_16")(H)
H = layers.Dense(32, activation="relu", name="dense_32")(H)

a_raw = layers.Dense(1, name="a_raw")(H)
a = layers.Lambda(lambda z: tf.cast(H_MAX, z.dtype) * tf.tanh(z),name="a")(a_raw)

policy = keras.Model(inputs=[features_in, dS_in, Z_T_in], outputs=a, name="policy_backbone")
policy.summary()

Model: "policy_backbone"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 features (InputLayer)       [(None, 252, 4)]             0         []                            
                                                                                                  
 gru_32 (GRU)                (None, 252, 32)              3648      ['features[0][0]']            
                                                                                                  
 gru_16 (GRU)                (None, 252, 16)              2400      ['gru_32[0][0]']              
                                                                                                  
 dense_32 (Dense)            (None, 252, 32)              544       ['gru_16[0][0]']              
                                                                                    

## 3.7) Symbolic rollout (within the Keras graph)

**What to do here (all tensor ops, no Python loops):**
1. **Positions:** `a = policy(features)` → shape `(B, n_steps, 1)`
2. **Turnover:** `a_prev` = shift of `a` with a zero at $(t=0)$;  
   `turnover = |a - a_prev|`
3. **Transaction cost:** get $(S_t)$ from features: `S_t = features[:,:,0] * S0` (then `[..., None]` to match dims);  
   `cost = GAMMA * sum_t (S_t * turnover)` → shape `(B, 1)` then squeeze to `(B,)`
4. **PnL:** `PnL = sum_t (a * dS)` → `(B,)`
5. **Terminal wealth:** `X = -Z_T + PnL - cost` → `(B,)`

**Monitoring tensors (for `add_metric`):**
- `mean_X = mean(X)`
- `mean_PnL = mean(PnL)`
- `mean_cost = mean(cost)`
- `turnover_mean = mean(turnover)` (averaged over batch & time)
- `bound_frac = mean( I[|a| >= 0.95*H_MAX] )`
- **Empirical tail stats on the batch** (for logging only):
  - $(k = \lceil (1-\alpha)$,$\text{batch\_size} \rceil)$
  - `X_sorted = sort(X)` (ascending)
  - `var_emp = X_sorted[k-1]`, `cvar_emp = mean(X_sorted[:k])`

In [8]:
# --------------------------
# 7) Rollout
# --------------------------

a_prev = tf.concat([tf.zeros_like(a[:, :1, :]), a[:, :-1, :]],axis=1) 
turnover = tf.abs(a - a_prev)

S_t = features_in[:, :, 0] * tf.cast(S0, tf.float32)   
S_t = tf.expand_dims(S_t, axis=-1)                     

cost = GAMMA * tf.reduce_sum(S_t * turnover, axis=1)

PnL = tf.reduce_sum(a * dS_in, axis=1)

X = -tf.expand_dims(Z_T_in, axis=-1) + PnL - cost
X = tf.squeeze(X, axis=-1)

mean_X = tf.reduce_mean(X)
mean_PnL = tf.reduce_mean(tf.squeeze(PnL, -1))
mean_cost = tf.reduce_mean(tf.squeeze(cost, -1))
turnover_mu = tf.reduce_mean(turnover)  # average over batch & time
bound_frac  = tf.reduce_mean(tf.cast(tf.abs(a) >= 0.95*H_MAX, tf.float32))

k = tf.cast(tf.math.ceil((1.0 - ALPHA) * tf.cast(tf.shape(X)[0], tf.float32)), tf.int32)
X_sorted = tf.sort(X)                # ascending = worse to better (since wealth; more negative = worse)
var_emp = X_sorted[k-1]
cvar_emp = tf.reduce_mean(X_sorted[:k])

## 3.8) RU CVaR head (loss layer)

**What to do here:**
- Add a custom layer **RUHead(α)** with a **trainable** scalar $(\tau)$ (initialized e.g. near the baseline VaR or a constant):
  $$
  \ell(X;\tau) \;=\; \frac{(\tau - X)^+}{1-\alpha} \;-\; \tau
  $$
- Properties:
  - Minimizing $( \mathbb{E}[\ell(X;\tau)] )$ over $(\tau)$ yields $(\tau = \text{VaR}_\alpha(X))$.
  - The minimized value equals $( \text{CVaR}_\alpha(X) )$.
- Pipe the rollout wealth \(X\) into RUHead to get per-sample losses $(\ell)$, then average.

**Optional auxiliary penalty (keep small):**
- $(\beta \cdot \text{mean}(|X|))$ to nudge overall MAE down, but **do not** let it dominate.

In [9]:
# --------------------------
# 8) CVaR head 
# --------------------------

BETA = 0.01

class RUHead(tf.keras.layers.Layer):
    def __init__(self, alpha,tau_init=-10.0,  **kwargs):
        super().__init__(**kwargs)
        self.alpha = float(alpha)
        self.tau_init = float(tau_init)

    def build(self, input_shape):
        self.tau = self.add_weight(
            name="tau",
            shape=(),
            initializer=tf.keras.initializers.Constant(self.tau_init),
            trainable=True,
            dtype=tf.float32
        )
    
    def call(self, X):
        X = tf.cast(X, self.tau.dtype)

        if X.shape.rank == 2 and X.shape[-1] == 1:
            X = tf.squeeze(X, axis=-1)
            
        hinge = tf.nn.relu(self.tau - X)
        ell = hinge / (1.0 - self.alpha) - self.tau
        self.add_metric(tf.identity(self.tau), name="tau", aggregation="mean")
        return ell


def identify_loss(y_true, y_pred):
    return y_pred

ru = RUHead(alpha=ALPHA, name="ru_head")
ell = ru(X)
mae_penalty = BETA * tf.reduce_mean(tf.abs(X))   

## 3.9) Compose the training model

**What to do here:**
- Create `loss_model = Model([features_in, dS_in, Z_T_in] → RUHead(X))`.
- Attach `add_metric(...)` for the monitors from §6 (mean_X, mean_PnL, mean_cost, turnover_mean, bound_frac, var_emp, cvar_emp) and also expose `tau` from RUHead.
- **Compile** with Adam (`learning_rate = LR`, optional `clipnorm=1.0`).
- Use an **identity loss** (`return y_pred`) because the RUHead already outputs $(\ell)$.


In [10]:
# --------------------------
# 9) Loss Model
# --------------------------

loss_model = keras.Model(inputs=[features_in, dS_in, Z_T_in],
                         outputs=ell,
                         name="loss")

loss_model.add_loss(mae_penalty)

loss_model.add_metric(mean_X, name="mean_X", aggregation="mean")
loss_model.add_metric(mean_PnL, name="mean_PnL", aggregation="mean")
loss_model.add_metric(mean_cost, name="mean_Cost", aggregation="mean")
loss_model.add_metric(turnover_mu, name="turnover_mean", aggregation="mean")
loss_model.add_metric(bound_frac, name="bound_frac", aggregation="mean")
loss_model.add_metric(cvar_emp, name="cvar_emp", aggregation="mean")
loss_model.add_metric(var_emp, name="var_emp", aggregation="mean")

mean_abs_X = tf.reduce_mean(tf.abs(X))
loss_model.add_metric(mean_abs_X, name="mean_abs_X", aggregation="mean")

loss_model.compile(optimizer=keras.optimizers.Adam(learning_rate = LR,
                                                   clipnorm=1.0),
                                                   loss=identify_loss,
                                                   )    

## 3.10) Training loop & callbacks

**What to do here:**
- Set callbacks:
  - `ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5)`
  - `EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)`
  - `ModelCheckpoint(..., monitor="val_cvar_emp", mode="min")` to keep best tails
  - `TerminateOnNaN()`
- `fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=cbs)`
- **How to read logs:**
  - `loss / val_loss` ≈ batch mean of RU loss (proxy for CVaR)
  - `tau` should settle near empirical VaR
  - `cvar_emp` (val) should **decrease** vs baseline from §4
  - `mean_cost`, `turnover_mean` give realism signals
  - `bound_frac` near zero (not slamming into the clamp)

In [11]:
# --------------------------
# 10) Training
# --------------------------

cbs = [
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min",
        factor=0.5, patience=5, min_lr=1e-5, verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss", mode="min",
        patience=12, restore_best_weights=True, min_delta=1e-3, verbose=1
    ),
    # save weights for the best tail (CVaR)
    keras.callbacks.ModelCheckpoint(
        filepath=str(RESULTS_DIR / "best_tail_by_cvar.weights.h5"),
        monitor="val_cvar_emp", mode="max",
        save_best_only=True, save_weights_only=True, verbose=1
    ),
    # optional: also save weights for best VaR (tau)
    keras.callbacks.ModelCheckpoint(
        filepath=str(RESULTS_DIR / "best_tail_by_var.weights.h5"),
        monitor="val_tau", mode="max",
        save_best_only=True, save_weights_only=True, verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=str(RESULTS_DIR / "best_center_by_absX.weights.h5"),
        monitor="val_mean_abs_X", mode="min",
        save_best_only=True, save_weights_only=True, verbose=1
    ),
    keras.callbacks.TerminateOnNaN(),
]

history = loss_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=cbs,
    verbose=1
)

Epoch 1/200
20/20 [==============================] - ETA: 0s - loss: 72.1091 - tau: -10.0095 - mean_X: -14.8970 - mean_PnL: -0.0501 - mean_Cost: 0.1039 - turnover_mean: 0.0082 - bound_frac: 0.0000e+00 - cvar_emp: -33.1711 - var_emp: -26.9874 - mean_abs_X: 14.9985
Epoch 1: val_cvar_emp improved from -inf to -27.28801, saving model to results\best_tail_by_cvar.weights.h5

Epoch 1: val_tau improved from -inf to -10.02001, saving model to results\best_tail_by_var.weights.h5

Epoch 1: val_mean_abs_X improved from inf to 14.72049, saving model to results\best_center_by_absX.weights.h5
20/20 [==============================] - 10s 367ms/step - loss: 72.1091 - tau: -10.0095 - mean_X: -14.8970 - mean_PnL: -0.0501 - mean_Cost: 0.1039 - turnover_mean: 0.0082 - bound_frac: 0.0000e+00 - cvar_emp: -33.1711 - var_emp: -26.9874 - mean_abs_X: 14.9985 - val_loss: 65.9077 - val_tau: -10.0200 - val_mean_X: -14.6509 - val_mean_PnL: 0.1813 - val_mean_Cost: 0.1089 - val_turnover_mean: 0.0086 - val_bound_frac:

KeyboardInterrupt: 

## 3.11) Global validation snapshot

**What to do here:**
- Build a lightweight `eval_model` that outputs **[X, cost, turnover, PnL]** given inputs (reuses the trained backbone + rollout).
- Run it on **all** validation paths (large batch; no shuffle).
- Compute and print:
  - `mean(X)` vs baseline mean of `-Z_T`
  - `VaR_α(X)` and `CVaR_α(X)` vs baseline
  - `mean(cost)` and **average total turnover per path** (sum over time, average over paths)
- Optionally store arrays for Notebook 4 plots.

In [ ]:
# --------------------------
# 11) Global Validation
# --------------------------

# Collect full validation set (inputs + targets)
xb_full = {"features": [], "dS": [], "Z_T": []}
yb_full = []

for xb, yb in val_ds:   # iterate batches
    for k in xb_full: 
        xb_full[k].append(xb[k].numpy())
    yb_full.append(yb.numpy())

# Concatenate into single arrays
for k in xb_full:
    xb_full[k] = np.concatenate(xb_full[k], axis=0)
yb_full = np.concatenate(yb_full, axis=0)


eval_model = keras.Model(
    inputs=[features_in, dS_in, Z_T_in],
    outputs=[X, cost, turnover, PnL]
)

X_val, cost_val, turnover_val, PnL_val = eval_model.predict(
    [xb_full["features"], xb_full["dS"], xb_full["Z_T"]],
    batch_size=1024, 
    verbose=1
)

N = X_val.shape[0]

# Mean
mean_XG = np.mean(X_val)

# VaR at 90% (10th percentile)
k = int(np.ceil((1 - ALPHA) * N))
X_sorted = np.sort(X_val)
VaR_90 = X_sorted[k-1]

# CVaR at 90% (average of worst 10%)
CVaR_90 = np.mean(X_sorted[:k])

# Mean cost
mean_cost = np.mean(cost_val)

# Total turnover per path (sum across timesteps, then average over paths)
turnover_total = np.mean(np.sum(turnover_val, axis=1))

X_zero = -xb_full["Z_T"]  # wealth is just -Z_T

# Zero-hedge baseline
X0_sorted = np.sort(X_zero)
mean_baseline = np.mean(X_zero)


var_baseline = X0_sorted[k-1]

hedged_mae = np.mean(np.abs(X_val))             # from eval_model outputs
baseline_mae = np.mean(np.abs(-xb_full["Z_T"])) # zero-hedge
print(f"MAE: model {hedged_mae:.3f} vs baseline {baseline_mae:.3f}")

print("=== Global Validation Evaluation ===")
print(f"Mean(XG): {mean_XG:.2f}  vs baseline {mean_baseline}")
print(f"VaR_90 : {VaR_90:.2f}  vs baseline {var_baseline:.2f}")
print(f"CVaR_90: {CVaR_90:.2f}  vs baseline {val_cvar_baseline:.2f}")
print(f"Mean Cost: {mean_cost:.4f}")
print(f"Avg Turnover: {turnover_total:.4f}")

## 3.12) Test set evaluation & saving artifacts

**What to do here:**
- Repeat §11 on `test_ds` and report the same metrics.
- **Save** to `results/hedging_eval_test_keras.npz`, e.g. keys:
  - `X` (terminal wealth), `Z_T`, `PnL`, `cost`, `turnover`, and possibly positions `a`
- These feed Notebook 4 for richer diagnostics/plots.

In [ ]:
# --------------------------
# 12) Testing
# --------------------------

xb_full = {"features": [], "dS": [], "Z_T": []}
yb_full = []

for xb, yb in test_ds:   # iterate batches
    for k in xb_full: 
        xb_full[k].append(xb[k].numpy())
    yb_full.append(yb.numpy())

for k in xb_full:
    xb_full[k] = np.concatenate(xb_full[k], axis=0)
yb_full = np.concatenate(yb_full, axis=0)

X_test, cost_test, turnover_test, PnL_test = eval_model.predict(
    [xb_full["features"], xb_full["dS"], xb_full["Z_T"]],
    batch_size=1024,
    verbose=1
)

N = X_test.shape[0]

mean_XG = np.mean(X_test)

k = int(np.ceil((1 - ALPHA) * N))
X_sorted = np.sort(X_test)
VaR_90 = X_sorted[k-1]
CVaR_90 = np.mean(X_sorted[:k])

mean_cost = np.mean(cost_test)
turnover_total = np.mean(np.sum(turnover_test, axis=1))

print("=== Global Test Evaluation ===")
print(f"Mean(X): {mean_XG:.2f}  vs baseline {mean_baseline:.2f}")
print(f"VaR_90 : {VaR_90:.2f}  vs baseline {var_baseline:.2f}")
print(f"CVaR_90: {CVaR_90:.2f}  vs baseline {val_cvar_baseline:.2f}")
print(f"Mean Cost: {mean_cost:.4f}")
print(f"Avg Turnover: {turnover_total:.4f}")

In [ ]:
# --------------------------
# 13) SAVE
# --------------------------
# Save V_T (hedged terminal portfolio) and Z_T (payoff) to NPZ
#   X_test, cost_test, turnover_test, PnL_test = eval_model.predict(...)
#   xb_full["Z_T"] from building the test set

V_T_test = (PnL_test.reshape(-1) - cost_test.reshape(-1)).astype(np.float32)  # (N,)
Z_T_test = xb_full["Z_T"].reshape(-1).astype(np.float32)                      # (N,)

# Sanity check
assert V_T_test.shape == Z_T_test.shape, f"Shape mismatch: {V_T_test.shape} vs {Z_T_test.shape}"

out_path = RESULTS_DIR / "hedging_eval_test_keras_5.npz"
np.savez_compressed(out_path, V_T=V_T_test, Z_T=Z_T_test)
print(f"Saved: {out_path}  (V_T {V_T_test.shape}, Z_T {Z_T_test.shape})")

In [ ]:
# loss_model.load_weights(RESULTS_DIR / "best_tail_by_cvar.weights.h5")

## 3.13) Interpreting results

**What good looks like (directionally):**
- `CVaR_α(X)` **much higher** (less negative) than baseline \( \text{CVaR}_\alpha(-Z_T) \)  
  (remember: more negative = worse loss; moving toward zero is improvement)
- `VaR_α(X)` improved similarly
- `mean(X)` may stay negative (you’re minimizing tail, not MSE); small positive is a bonus
- `mean_cost` and `turnover` within realistic bounds (gives credibility)
- `bound_frac` small (not exploiting the clamp edge)

**If it looks too good to be true:**
- Check no look-ahead in features (§2)
- Ensure `dS` aligns with time \(t\) features
- Confirm `Z_T` shape is 1D and used only in `X = -Z_T + ...`

---

## 3.14) Hyperparameter guide (what changes what)

- **`ALPHA` (tail level):**  
  Higher (e.g., 0.95) ⇒ focuses on *extreme* worst tail, typically stronger tail protection, possibly larger mean error.  
  Lower (e.g., 0.80) ⇒ softer tail, may improve average metrics but weaker extreme-loss control.
- **`GAMMA` (cost):**  
  Higher ⇒ discourages turnover, reduces cost & noise, might leave some risk unhedged.  
  Lower ⇒ more active hedging, better fit, but higher costs / overfitting risk.
- **`H_MAX` (clamp):**  
  Lower ⇒ policies stay small; safer but under-powered.  
  Higher ⇒ more capacity; watch `bound_frac`.
- **Backbone size (GRU widths):**  
  Larger ⇒ more expressive; use dropout or stronger in case of overfitting.
- **`LR`, `BATCH`:**  
  Tail objectives benefit from moderately large batches (more stable tail estimate). Too small batches produce noisy VaR/CVaR.